In [2]:
import numpy as np
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth

from planet import api
from planet.api import filters
from sys import stdout
import json
import io
import tempfile

from datetime import date, datetime, timedelta
import time
import os
import pathlib
import fnmatch

In [3]:
geojson_geometry = {
    "type": "Point",
    "coordinates": [
         -118.174016,35.473354
        ]
}

# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

In [4]:
# Generating time parameters
today = datetime.isoformat(datetime.utcnow())+'Z'#(datetime.today())
start_date = datetime.isoformat(datetime.utcnow() - timedelta(7)) + 'Z'

In [5]:
# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": start_date,
    "lte": today
  }
}

# only get images which have <10% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.1
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [6]:
# Generating Order

# API Key stored as an env variable
PL_API_KEY = 'XXXXXXXXXXXXXXXX'
PLANET_API_KEY = PL_API_KEY #os.getenv('PL_API_KEY')
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}

item_type = "PSScene4Band"

# API request object
search_request = {
    "interval": "day",
    "item_types": [item_type],
    "asset_types" : "analytic_sr",
    "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
#     'https://api.planet.com/data/v2',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

print(json.dumps(search_result.json(), indent=1))

{
 "_links": {
  "_first": "https://api.planet.com/data/v1/searches/ce910a107d8a47b5a7a889a6c8447141/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D",
  "_next": "https://api.planet.com/data/v1/searches/ce910a107d8a47b5a7a889a6c8447141/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIwLTAyLTE1VDIzOjU3OjQxLjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjAwMjE1XzE4MTkyOF8wZjRlIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319",
  "_self": "https://api.planet.com/data/v1/searches/ce910a107d8a47b5a7a889a6c8447141/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D"
 },
 "features": [
  {
   "_links": {
    "

In [7]:
# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]

# For demo purposes, just grab the first image ID
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )

# List of asset types available for this particular satellite image
#print(result.json().keys())

In [8]:
# Parse out useful links
links = result.json()[u"analytic_sr"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

# Request activation of the 'visual' asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )

In [9]:
# # Activation Status if needed
# activation_status_result = \
#   requests.get(
#     self_link,
#     auth=HTTPBasicAuth(PLANET_API_KEY, '')
#   )

## Ordering from V2
* https://api.planet.com/compute/ops/orders/v2
* https://github.com/planetlabs/notebooks/blob/d92943e82a87de7653452a99084c4a16a877babf/jupyter-notebooks/orders/tools_and_toolchains.ipynb

In [10]:
# define products part of order
single_product = [
    {
      "item_ids": [id0],#["20151119_025740_0c74"],
      "item_type": "PSScene4Band",
      "product_bundle": "analytic_sr"
    }
]

In [11]:
# define helpful functions for submitting, polling, and downloading an order
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if not response.ok:
        raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

def poll_for_success(order_url, auth, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(10)
        
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join('data', n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))


### Clip
<a id='clip'></a>

Clipping is likely the most common tool that will be used. It allows us to only download the pixels we are interested in.

In [12]:
clip_aoi = {
    "type":"Polygon",
    "coordinates":[[[-118.178016, 35.469354],
                    [-118.178016, 35.477354],
                    [-118.17001599999999, 35.477354],
                    [-118.17001599999999, 35.469354],
                    [-118.178016, 35.469354]]]
}

# define the clip tool
clip = {
    "clip": {
        "aoi": clip_aoi
    }
}

# create an order request with the clipping tool
request_clip = {
  "name": "just clip",
  "products": single_product,
  "tools": [clip]
}

In [13]:
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

order_url = place_order(request_clip, auth)

<Response [202]>
4d299202-044a-49db-8dcb-367e7f5c19ab


In [14]:
# # Pulling specific image
# #r = requests.get(order_url, auth=auth)
# response = requests.get(order_url, auth=auth).json()
# #response = r.json()
# results = response['_links']['results']
# results_name = [r['name'] for r in results]
# pattern = '*SR_clip.tif'
# results_sr = fnmatch.filter(results_name, pattern)

KeyError: 'results'

In [15]:
# allow for caching so we don't always run clip
run_clip = True

# clip_img_file = results_sr[0]
# if os.path.isfile(clip_img_file): run_clip = False

In [16]:
if run_clip:
    clip_order_url = place_order(request_clip, auth)
    poll_for_success(clip_order_url, auth)
#     downloaded_clip_files = download_order(clip_order_url, auth)
#     clip_img_file = next(downloaded_clip_files[d] for d in downloaded_clip_files
#                      if d.endswith('_SR.tif'))
#downloaded_clip_files = download_order(clip_order_url, auth)
#clip_img_file

<Response [202]>
8525aef9-1bc9-4b9d-bbdf-cb31a41305a7
queued
queued
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
success


In [17]:
download_order(clip_order_url, auth)

<Response [200]>
5 items to download
downloading 8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/manifest.json to data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/manifest.json
downloading 8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_DN_udm_clip.tif to data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_DN_udm_clip.tif
downloading 8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_metadata_clip.xml to data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_metadata_clip.xml
downloading 8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_metadata.json to data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_metadata.json
downloading 8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_SR_clip.tif to data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_SR_clip.tif


{'8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/manifest.json': PosixPath('data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/manifest.json'),
 '8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_DN_udm_clip.tif': PosixPath('data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_DN_udm_clip.tif'),
 '8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_metadata_clip.xml': PosixPath('data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_metadata_clip.xml'),
 '8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_metadata.json': PosixPath('data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_metadata.json'),
 '8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_SR_clip.tif': PosixPath('data/8525aef9-1bc9-4b9d-bbdf-cb31a41305a7/1/files/20200219_181635_1039_3B_AnalyticMS_SR_clip.tif')}